In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import matplotlib as plt 
import numpy as np
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.cross_validation import KFold,cross_val_score
import itertools
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")

c:\python27\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
train.columns

Index([u'PassengerId', u'Survived', u'Pclass', u'Name', u'Sex', u'Age',
       u'SibSp', u'Parch', u'Ticket', u'Fare', u'Cabin', u'Embarked'],
      dtype='object')

In [4]:
train.head(n=3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [5]:
pd.isnull(train["Age"])
train["Age"]=train.fillna(train["Age"].median())
train["Embarked"]=train["Embarked"].fillna("S")
train.loc[train["Embarked"]=="S","Embarked"]=1
train.loc[train["Embarked"]=="Q","Embarked"]=2
train.loc[train["Embarked"]=="C","Embarked"]=3

In [6]:
train.head(n=5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,1,1,0,A/5 21171,7.2500,NaN,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,2,1,0,PC 17599,71.2833,C85,3
2,3,1,3,"Heikkinen, Miss. Laina",female,3,0,0,STON/O2. 3101282,7.9250,NaN,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,4,1,0,113803,53.1000,C123,1
4,5,0,3,"Allen, Mr. William Henry",male,5,0,0,373450,8.0500,NaN,1


In [7]:
train.loc[train["Sex"]=="male","Sex"]=0
train.loc[train["Sex"]=="female","Sex"]=1
train.head(n=5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,1,1,0,A/5 21171,7.2500,NaN,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,2,1,0,PC 17599,71.2833,C85,3
2,3,1,3,"Heikkinen, Miss. Laina",1,3,0,0,STON/O2. 3101282,7.9250,NaN,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,4,1,0,113803,53.1000,C123,1
4,5,0,3,"Allen, Mr. William Henry",0,5,0,0,373450,8.0500,NaN,1


## Plots on Data

## Regressor

In [8]:
predictors = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]

In [9]:
alg=LinearRegression()
kf = KFold(train.shape[0], n_folds=3, random_state=1)

In [10]:
predictions=[]
for tr,ts in kf:
    train_predictors=(train[predictors].iloc[tr,:])
    train_target=(train["Survived"].iloc[tr])
    alg.fit(train_predictors,train_target)
    test_predictions=alg.predict(train[predictors].iloc[ts,:])
    predictions.append(test_predictions)

In [11]:
predictions = np.concatenate(predictions, axis=0)

In [12]:
predictions[predictions > 0.5]=1
predictions[predictions < 0.5]=0
accuracy=(sum(predictions[predictions==train["Survived"]])/len(predictions))*100
acc_str=str(accuracy)
print("Accuracy of the model is " + acc_str)

Accuracy of the model is 77.7777777778


c:\python27\lib\site-packages\ipykernel\__main__.py:3: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
  app.launch_new_instance()


### Logistic Regression

In [13]:
alg=LogisticRegression(random_state=1)
alg.fit(train[predictors],train["Survived"])
scores=cross_val_score(alg,train[predictors],train["Survived"],cv=3)
scores.mean()

0.78675645342312006

## Test Set

#pd.isnull(train["Age"])
#train["Age"]=train.fillna(train["Age"].median())
#train["Embarked"]=train["Embarked"].fillna("S")
#train.loc[train["Embarked"]=="S","Embarked"]=1
#train.loc[train["Embarked"]=="Q","Embarked"]=2
#train.loc[train["Embarked"]=="C","Embarked"]=3
#train.loc[train["Sex"]=="male","Sex"]=0
#train.loc[train["Sex"]=="female","Sex"]=1

In [14]:
test["Age"]=test.fillna(train["Age"].median())
test["Embarked"]=test["Embarked"].fillna("S")
test["Fare"]=test["Fare"].fillna(test["Fare"].median())
test.loc[test["Embarked"]=="S","Embarked"]=1
test.loc[test["Embarked"]=="Q","Embarked"]=2
test.loc[test["Embarked"]=="C","Embarked"]=3
test.loc[test["Sex"]=="male","Sex"]=0
test.loc[test["Sex"]=="female","Sex"]=1

In [17]:
alg=LogisticRegression(random_state=1)
alg.fit(train[predictors],train["Survived"])
predictions=alg.predict(test[predictors])

Index([u'PassengerId', u'Pclass', u'Name', u'Sex', u'Age', u'SibSp', u'Parch',
       u'Ticket', u'Fare', u'Cabin', u'Embarked'],
      dtype='object')